<a href="https://colab.research.google.com/github/kridtapon/Bollinger-Regression-Momentum-Strategy/blob/main/Bollinger_Regression_Momentum_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.2 MB/s eta 0:00:00
  Created wheel for vectorbt: filename=vectorbt-0.26.2-py3-none-any.whl size=527326 sha256=6dcb252024bd2a7d924f37ea6d4b52a205a6adf51d3940e3231026fbfeebe57c
  Stored in directory: /root/.cache/pip/wheels/35/a3/d2/ae1a8cbf659fffc7d1890123df1a7fa2e7f6df587c1dca71cd
Successfully built vectorbt
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
     

In [33]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression

def calculate_bollinger_bands(df, window=20, num_std=2):
    """Calculate Bollinger Bands manually."""
    rolling_mean = df['Close'].rolling(window=window).mean()
    rolling_std = df['Close'].rolling(window=window).std()

    # Upper and lower Bollinger Bands
    df['BB_Upper'] = rolling_mean + (num_std * rolling_std)
    df['BB_Lower'] = rolling_mean - (num_std * rolling_std)
    return df

def calculate_linear_regression_slope(series, window=10):
    """Calculate the Linear Regression Slope."""
    slopes = []
    for i in range(len(series)):
        if i < window - 1:
            slopes.append(np.nan)
        else:
            y = series[i - window + 1:i + 1].values
            x = np.arange(len(y)).reshape(-1, 1)
            model = LinearRegression().fit(x, y)
            slopes.append(model.coef_[0])
    return pd.Series(slopes, index=series.index)

def main():
    # Define the stock symbol and time period
    symbol = '^SPX'
    start_date = '2014-12-17'
    end_date = '2024-12-17'

    # Download the data
    df = yf.download(symbol, start=start_date, end=end_date)
    df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    df.ffill(inplace=True)

    # Calculate Bollinger Bands
    df = calculate_bollinger_bands(df, window=20, num_std=1.5)

    # Calculate Linear Regression Slope
    df['LR_Slope'] = calculate_linear_regression_slope(df['Close'], window=10)
    df['LR_Slope_Yesterday'] = df['LR_Slope'].shift(1)
    df['LR_Slope_Before_Yesterday'] = df['LR_Slope'].shift(2)

    # Add day of the week
    df['DayOfWeek'] = df.index.to_series().dt.dayofweek  # Monday=0, Sunday=6

    # Define entry signal
    df['Entry'] = (
        (df['Close'] < df['Open']) &  # Close is less than Open
        (df['LR_Slope'] > df['LR_Slope_Yesterday']) &  # Today's slope is higher than yesterday's
        (df['LR_Slope_Yesterday'] < df['LR_Slope_Before_Yesterday'])  # Yesterday's slope is lower than the day before
    )

    # Define exit signal
    df['Exit'] = (
        (df['Close'] > df['BB_Upper']) & # Close above the upper Bollinger Band
        (df['LR_Slope'] < df['LR_Slope_Yesterday']) &  # Today's slope is higher than yesterday's
        (df['LR_Slope_Yesterday'] > df['LR_Slope_Before_Yesterday'])
    )

    # Convert signals to boolean arrays
    entries = df['Entry'].to_numpy()
    exits = df['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    # Display performance metrics
    print(portfolio.stats())

    # Plot equity curve
    portfolio.plot().show()

if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2014-12-17 00:00:00
End                           2024-12-16 00:00:00
Period                                       2516
Start Value                              100000.0
End Value                           381695.702478
Total Return [%]                       281.695702
Benchmark Return [%]                   201.759164
Max Gross Exposure [%]                      100.0
Total Fees Paid                      14574.495654
Max Drawdown [%]                        15.742357
Max Drawdown Duration                       258.0
Total Trades                                   35
Total Closed Trades                            35
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            91.428571
Best Trade [%]                          26.112423
Worst Trade [%]                         -1.700611
Avg Winning Trade [%]                    4.459389
Avg Losing Trade [%]                    -0.932709


In [26]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from itertools import product  # For parameter combinations

def calculate_bollinger_bands(df, window=20, num_std=2):
    """Calculate Bollinger Bands manually."""
    rolling_mean = df['Close'].rolling(window=window).mean()
    rolling_std = df['Close'].rolling(window=window).std()
    df['BB_Upper'] = rolling_mean + (num_std * rolling_std)
    df['BB_Lower'] = rolling_mean - (num_std * rolling_std)
    return df

def calculate_linear_regression_slope(series, window=3):
    """Calculate the Linear Regression Slope."""
    slopes = []
    for i in range(len(series)):
        if i < window - 1:
            slopes.append(np.nan)
        else:
            y = series[i - window + 1:i + 1].values
            x = np.arange(len(y)).reshape(-1, 1)
            model = LinearRegression().fit(x, y)
            slopes.append(model.coef_[0])
    return pd.Series(slopes, index=series.index)

def run_backtest(df, bb_window, bb_num_std, lr_window):
    """Run backtest for given parameter values."""
    # Calculate Bollinger Bands
    df = calculate_bollinger_bands(df.copy(), window=bb_window, num_std=bb_num_std)

    # Calculate Linear Regression Slope
    df['LR_Slope'] = calculate_linear_regression_slope(df['Close'], window=lr_window)
    df['LR_Slope_Yesterday'] = df['LR_Slope'].shift(1)
    df['LR_Slope_Before_Yesterday'] = df['LR_Slope'].shift(2)

    # Define entry and exit signals
    df['Entry'] = (
        (df['Close'] < df['Open']) &
        (df['LR_Slope'] > df['LR_Slope_Yesterday']) &
        (df['LR_Slope_Yesterday'] < df['LR_Slope_Before_Yesterday'])
    )
    df['Exit'] = (
        (df['Close'] > df['BB_Upper']) &
        (df['LR_Slope'] < df['LR_Slope_Yesterday']) &
        (df['LR_Slope_Yesterday'] > df['LR_Slope_Before_Yesterday'])
    )

    # Convert signals to boolean arrays
    entries = df['Entry'].to_numpy()
    exits = df['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    return portfolio.stats()

def optimize_parameters():
    """Grid search to find the best parameters."""
    # Define ranges for parameters
    bb_window_range = [10, 20, 30, 50]
    bb_num_std_range = [1.5, 2, 2.5, 3]
    lr_window_range = [3, 5, 7, 10]

    # Download stock data
    symbol = '^SPX'
    start_date = '2019-01-01'
    end_date = '2024-01-01'
    df = yf.download(symbol, start=start_date, end=end_date)
    df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    df.ffill(inplace=True)

    # Grid search
    best_params = None
    best_metric = -np.inf  # Initialize to very low Total Return or metric
    results = []

    for bb_window, bb_num_std, lr_window in product(bb_window_range, bb_num_std_range, lr_window_range):
        stats = run_backtest(df, bb_window, bb_num_std, lr_window)
        sharpe = stats['Total Return [%]']  # Use Total Return as optimization metric
        results.append((bb_window, bb_num_std, lr_window, sharpe))
        print(f"BB_Window: {bb_window}, BB_Num_Std: {bb_num_std}, LR_Window: {lr_window}, Total Return [%]: {sharpe}")

        # Update best parameters if Sharpe improves
        if sharpe > best_metric:
            best_metric = sharpe
            best_params = (bb_window, bb_num_std, lr_window)

    # Display the best parameters
    print("\nBest Parameters:")
    print(f"Bollinger Bands Window: {best_params[0]}, Num Std Dev: {best_params[1]}, LR Window: {best_params[2]}")
    print(f"Best Sharpe Ratio: {best_metric}")

    return results, best_params

if __name__ == "__main__":
    results, best_params = optimize_parameters()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



BB_Window: 10, BB_Num_Std: 1.5, LR_Window: 3, Total Return [%]: 23.566759978373998


/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



BB_Window: 10, BB_Num_Std: 1.5, LR_Window: 5, Total Return [%]: 81.6000669968999


/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



...

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



BB_Window: 50, BB_Num_Std: 3, LR_Window: 5, Total Return [%]: 70.4176455878366


/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



BB_Window: 50, BB_Num_Std: 3, LR_Window: 7, Total Return [%]: 73.62550230064188
BB_Window: 50, BB_Num_Std: 3, LR_Window: 10, Total Return [%]: 63.99702933628429

Best Parameters:
Bollinger Bands Window: 20, Num Std Dev: 1.5, LR Window: 10
Best Sharpe Ratio: 140.85994075637697


/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

